In [ ]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel


import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets

import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import plotting
#import cv2
from torch.nn.utils import weight_norm
import scipy.misc
#%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# to prevent opencv from initializing CUDA in workers
#torch.randn(8).cuda()
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

def gallery(array, ncols=3):
    nindex, height, width, intensity = array.shape
    nrows = nindex//ncols
    assert nindex == nrows*ncols
    # want result.shape = (height*nrows, width*ncols, intensity)
    result = (array.reshape((nrows, ncols, height, width, intensity))
              .swapaxes(1,2)
              .reshape((height*nrows, width*ncols, intensity)))
    return result

def save_samples():
    img_bhwc = netG(noise).data.cpu()[0:100].view(100,1,28,28).expand(100,3,28,28)
    img_bhwc = img_bhwc.permute(0,2,3,1).numpy()
    array = img_bhwc.copy()
    result = gallery(array,10)*.5+.5
    scipy.misc.imsave('outfile_noent_1.jpg', result)


In [ ]:
count = 400
learning_rate = .0001
batch_size = 500
unlabeled_weight = 1
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)


In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       #transforms.Normalize((0.1307,), (0.3081,)  )
                   ])),
    batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       #transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=False)

trainx = train_loader.dataset.train_data.float().view(-1,1,28,28)/255
trainy = train_loader.dataset.train_labels


In [ ]:
nc = 1
ndf = 64
ncl = 100
class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            #nn.Conv2d(ndf, ndf, 3, 1, 1, bias=False),
            #nn.BatchNorm2d(ndf),
            #nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 2, ndf * 2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 3, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 4, 3, 1, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 4, ncl, 4, 1, 0, bias=False),
        )

    def forward(self, input):
        output = self.main(input)
        return F.softmax(output.view(-1, ncl))


netD = _netD()
netD.cuda()
checkpoint = torch.load('model')
netD.load_state_dict(checkpoint)


In [ ]:
posterior = torch.zeros(60000,100)
for i in range(0, trainx.size(0), batch_size):
    input = trainx[i:i+batch_size]
    input = Variable(input.cuda())
    posterior[i:i+batch_size] = netD(input).data.cpu()

assignment_x = posterior.max(1)[1]
intersect = torch.zeros(100,10)

for i in range(0,100):
    for j in range(0,10):
        intersect[i][j] = ((assignment_x==i)*(trainy==j)).sum()

accuracy = intersect.max(1)[0].sum()/trainx.size(0)
print(accuracy)

In [ ]:
ncl = 160
class DynamicGNoiseConv(nn.Module):
    def __init__(self, shape, std=0.05):
        super().__init__()
        self.noise = Variable(torch.zeros(1,1,shape,shape).cuda())
        self.std   = std
        
    def forward(self, x):
        if not training: return x
        self.noise.data.normal_(0, std=self.std)      
        return x + self.noise.expand_as(x)

class DynamicGNoise(nn.Module):
    def __init__(self, shape, std=0.1):
        super().__init__()
        self.noise = Variable(torch.zeros(batch_size,shape).cuda())
        self.std   = std
        
    def forward(self, x):
        if not training: return x
        self.noise.data.normal_(0, std=self.std)      
        return x + self.noise
    
    
ins = [784,1000,500,250,250,250,10]
sigma = [.3,.5,.5,.5,.5,.5]
  

class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()

        self.conv1 = nn.Conv2d(1,50,5)
        self.conv1.weight.data.normal_(0,.05)
        self.conv1.bias.data.fill_(0)
        self.conv1 = weight_norm(self.conv1)
        self.conv2 = nn.Conv2d(50,50,5)
        self.conv2.weight.data.normal_(0,.05)
        self.conv2.bias.data.fill_(0)
        self.conv2 = weight_norm(self.conv2)
        self.linear1 = nn.Linear(50*64,ncl)
        self.linear1.weight.data.normal_(0,.05)
        self.linear1.bias.data.fill_(0)
        self.linear1 = weight_norm(self.linear1)

        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(input=x, kernel_size=2, stride=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.view(-1,50*64)
        x = self.linear1(x)
        return x

netD = _netD()
netD.cuda()
#netD.apply(weights_init);

optimizerC = optim.Adam(netD.parameters(), lr=learning_rate, betas=(.5, .999))
phi_all = torch.randn(6000,ncl).cuda()
phi_all = Variable(phi_all)



def initialize(x_unl):
    global saved_g1, saved_g2
    netD.eval()
    x = Variable(x_unl.cuda())
    x_new = netD.conv1(x).data
    m = x_new.mean(3).mean(2).mean(0)
    inv_stdv = 1/(x_new**2).mean(3).mean(2).mean(0).sqrt().view(-1,1,1,1)
    netD.conv1.weight_g.data.copy_(netD.conv1.weight_g.data*inv_stdv)
    netD.conv1.bias.data.copy_(-m*inv_stdv.squeeze())
    x = netD.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(input=x, kernel_size=2, stride=2)

    x_new = netD.conv2(x).data
    m = x_new.mean(3).mean(2).mean(0)
    inv_stdv = 1/(x_new**2).mean(3).mean(2).mean(0).sqrt().view(-1,1,1,1)
    netD.conv2.weight_g.data.copy_(netD.conv2.weight_g.data*inv_stdv)
    netD.conv2.bias.data.copy_(-m*inv_stdv.squeeze())
    x = netD.conv2(x)
    x = F.relu(x)
    x = x.view(-1,50*64)
    
    x_new = netD.linear1(x).data
    m = x_new.mean(0)
    inv_stdv = 1/(x_new**2).mean(0).sqrt().view(-1,1)
    netD.linear1.weight_g.data.copy_(netD.linear1.weight_g.data*inv_stdv)
    netD.linear1.bias.data.copy_(-m*inv_stdv.squeeze())
    
    
    saved_g1 = netD.conv1.weight_g.data.clone()
    saved_g2 = netD.conv2.weight_g.data.clone()

In [ ]:
def evaluate(assignment_x, trainy):    
    intersect = torch.zeros(max_class,10)
    i = 0                           
    for i in range(0, max_class):
        for j in range(0,10):
            intersect[i][j] = ((assignment_x==i)*(trainy==j)).sum()
        i+=1    
    accuracy = intersect.max(1)[0].sum()/intersect.sum()
    return accuracy
        
 
def compress():
    global assignment_x, max_class, emp_sum_k, count_k, mean_k
    un_labs = torch.from_numpy(np.unique(assignment_x.cpu().numpy())).squeeze().long().cuda()
    reversed_index = torch.zeros(un_labs.max()+1).long().cuda()
    max_class = un_labs.size(0)
    range1 = torch.from_numpy(np.asarray(range(0,max_class))).cuda()
    reversed_index.index_copy_(0, un_labs, range1)
    assignment_x = reversed_index.index_select(0,assignment_x.long())
    emp_sum_k[:max_class] = emp_sum_k.index_select(0, un_labs)
    count_k[:max_class] = count_k.index_select(0, un_labs)
    means_k[:max_class] = emp_sum_k[:max_class]/(count_k[:max_class].unsqueeze(1) + rho0/rho)

In [ ]:
def get_means(input, batchL):
    count_k1.zero_()
    emp_sum_k1.zero_()
    for i in range(0, input.size(0)):
        label = batchL[i]
        emp_sum_k1[label] += input[i]
        count_k1[label] += 1
    
    means_k[:max_class] = emp_sum_k1[:max_class]/(count_k1[:max_class].unsqueeze(1) + rho0/rho)
    return means_k.index_select(0, batchL.cuda())

def train_classifier(x_unl, batchL):
    global phi_all, output
    netD.train() 
    optimizerC.zero_grad()
    
    
    x_unl = Variable(x_unl.cuda())
    output = netD(x_unl)

    #Computation of loss_disc
    means = get_means(output.data, batchL)    
    
    z = output
    phi_all = Variable(phi_all.data)
    phi_all = torch.cat((output,phi_all),0)[0:6000]
    term3 = torch.mm(z, phi_all.t())
    term1 = (z**2).sum(1, keepdim=True).expand_as(term3)
    term2 = (phi_all**2).sum(1, keepdim=True).t().expand_as(term3)
    exponent = term1+term2-2*term3
    exponent = rho/2*exponent
    exponent = exponent*-1
    temp = exponent.exp()
    px_z_inv = temp.sum(1) 
    loss_disc = px_z_inv.log().mean() 
    means = Variable(means)
    loss_prior = rho/2*((output - means)**2).sum(1).mean() 
    
    loss = loss_disc + loss_prior
    loss.backward()
    
    optimizerC.step()
    return loss_disc.data[0], loss_prior.data[0]
    



In [ ]:
def collapsedGibbs(input, batchL):
    global max_class, probs
    #Get the means based on the current assignments
    #The mean is computed over the batch
    smallest_ind = count_k[:max_class].min(0)[1][0]
    if epoch<2 or max_class<100:
        numEp = 10
    else:
        numEp = 3
        
    for iter in range(0,numEp):
        count_k.zero_()
        emp_sum_k.zero_()
        for i in range(0, input.size(0)):
            label = batchL[i]
            if(label!=smallest_ind):
                emp_sum_k[label] += input[i]
                count_k[label] += 1

        means_k = emp_sum_k[:max_class]/(count_k[:max_class].unsqueeze(1) + rho0/rho)
        means_k[smallest_ind].fill_(1000000)

        term1 = torch.mm(input, means_k.t())
        term2 = (input**2).sum(1, keepdim = True)
        term3 = (means_k**2).sum(1, keepdim = True).t()
        dist = term2 + term3 - 2*term1
        dist = rho/2*dist*-1
        #print(dist.size())
        #print(count_k.size())
        #dist = dist + torch.log(count_k[:max_class].unsqueeze(0)+ 1)
        batchL = dist.max(1)[1]

    
    return batchL

In [ ]:
import time
import scipy.misc
eyeMat = Variable(torch.eye(batch_size).cuda())
rho = 1/160
rho0 = 1/300
rho_mean = rho*rho0/(rho+rho0)
alpha = 1
max_class = 300
means_k = torch.zeros(1000,ncl).cuda()
#Stores the number of points in each cluster
count_k = torch.zeros(1000).cuda()
#Stores the empirical mean of the points in each cluster
emp_sum_k = torch.zeros(1000,ncl).cuda()
#Assignment variable for each cluster
embeddings = torch.zeros(60000,ncl).cuda()
cmk = torch.zeros(batch_size, ncl).cuda()
probs = torch.zeros(1000).cuda()

loss_disc = torch.zeros(60000)
loss_prior = torch.zeros(60000)
trainy = trainy.cuda()
shuffle = torch.randperm(60000)
assignment_x = torch.zeros(60000).int()
for i in range(0,60000):
    assignment_x[i] = i%max_class
assignment_x = assignment_x.cuda()
assignment_x = assignment_x[shuffle.cuda()]
count_k1 = count_k.clone()
emp_sum_k1 = emp_sum_k.clone()


In [ ]:
for epoch in range(0,500):
    
    begin = time.time()
    lr = .0001
    
        
    if epoch==0:
        print(trainx.shape)
        initialize(trainx[:500]) # data based initialization

    if(max_class<20):
        break

    shuffle = torch.randperm(60000)
    trainx = trainx[shuffle]
    trainy = trainy[shuffle.cuda()]
    assignment_x = assignment_x[shuffle.cuda()]
    
    numBatches = 0
    for i in range(0, trainx.size(0), batch_size):
        numBatches +=1
        x_unl = trainx[i:i+batch_size]
        x_ass = assignment_x[i:i+batch_size].long()
        ld, lp = train_classifier(x_unl, x_ass)
        loss_disc[epoch] += ld
        loss_prior[epoch] += lp
        
        netD.conv1.weight_g.data.copy_(saved_g1)
        netD.conv2.weight_g.data.copy_(saved_g2)
        
    for i in range(0, trainx.size(0), batch_size):
        input = trainx[i:i+batch_size]
        input = Variable(input.cuda())
        embeddings[i:i+batch_size] = netD(input).data
    assignment_x = collapsedGibbs(embeddings, assignment_x.clone())
    
    
    compress()
    
    print('Evaluating test accuracy')
    accuracy = evaluate(assignment_x, trainy)
    print("Disc loss:%.4f, prior loss:%.4f, purity:%.4f"%(loss_disc[epoch], loss_prior[epoch], accuracy))

    